# 「單一」股票最佳擇時(Timing)因子選擇

In [ ]:
"""
因子優化模組使用範例
"""
import setup_path  # 設定 Python 路徑以引用 alanq 模組
from alanq.optimization import ParameterSpace, FactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor, AtrStopSellFactor
from alanq.data import StockDataManager

def example_optimization():
    """範例：單股票因子優化"""
    
    # 1. 準備資料
    print("正在下載資料...")
    data_manager = StockDataManager(["TSLA"], start_date="2020-01-01")
    df = data_manager.get_kl_pd("TSLA")
    
    # 2. 定義參數空間
    print("定義參數空間...")
    param_space = ParameterSpace()
    
    # 添加買入因子
    param_space.add_buy_factor(
        BreakoutBuyFactor,
        {'xd': [20, 30, 40, 50, 60], 'skip_days': [10, 15, 20]}
    )
    
    # 添加賣出因子
    param_space.add_sell_factor(
        BreakdownSellFactor,
        {'xd': [10, 15, 20, 25]}
    )
    
    param_space.add_sell_factor(
        AtrStopSellFactor,
        {'atr_n': [1.0, 1.5, 2.0, 2.5]}
    )
    
    
    # 查看總組合數
    total_combos = param_space.get_total_combinations()
    print(f"總組合數: {total_combos}")
    
    # 3. 執行優化
    print("\n開始優化...")
    # 定義自定義權重
    custom_weights = {
        '策略_總報酬率': 1,     
        '策略_Sharpe': 1,       
        '策略_最大回撤': 1,     
        '勝率': 1,              
        '盈虧比': 1,             
        # 其他指標權重為 0（不考慮）
    }

    # 使用自定義權重進行優化
    optimizer = FactorOptimizer(
        df=df,
        parameter_space=param_space,
        metric_weights=custom_weights,  # 傳入自定義權重
        initial_capital=1_000_000,
        show_progress=True
    )
    # optimizer = FactorOptimizer(
    #     df=df,
    #     parameter_space=param_space,
    #     initial_capital=1_000_000,
    #     show_progress=True
    # )
    
    best_config, results_df = optimizer.optimize()
    
    # 4. 查看結果
    if best_config:
        print(f"\n{'='*60}")
        print("最佳配置:")
        print(f"{'='*60}")
        print(f"買入因子: {best_config['buy_factors']}")
        print(f"賣出因子: {best_config['sell_factors']}")
        print(f"總得分: {best_config['總得分']:.2f}")
        
        print(f"\n{'='*60}")
        print("前 10 名結果:")
        print(f"{'='*60}")
        top_results = optimizer.get_top_n(10)
        display_cols = ['組合編號', '總得分', '策略_總報酬率', '策略_Sharpe', 
                       '策略_最大回撤', '勝率', '盈虧比']
        available_cols = [col for col in display_cols if col in top_results.columns]
        print(top_results[available_cols].to_string())
    else:
        print("優化失敗，沒有找到有效結果")
    
    return best_config, results_df

if __name__ == "__main__":
    best_config, results_df = example_optimization()

正在下載資料...
正在使用 yfinance 下載 1 檔股票資料...
------------------------------
已成功下載 1 檔股票資料
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
------------------------------
定義參數空間...
總組合數: 360

開始優化...
總共需要測試 360 種組合...


收集指標數據: 100%|██████████| 360/360 [00:38<00:00,  9.41it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.2000
  - 勝率: 0.2000
  - 盈虧比: 0.2000

最佳配置:
買入因子: [{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20, 'skip_days': 10}]
賣出因子: [{'class': <class 'alanq.factors.timing.atr_stop_sell_factor.AtrStopSellFactor'>, 'atr_n': 1.0}]
總得分: 60.25

前 10 名結果:
          總得分   策略_總報酬率  策略_Sharpe   策略_最大回撤
4   60.254224  8.017246   0.564109 -0.736322
5   60.254224  8.017246   0.564109 -0.736322
6   60.254224  8.017246   0.564109 -0.736322
7   60.254224  8.017246   0.564109 -0.736322
28  60.254224  8.017246   0.564109 -0.736322
29  60.254224  8.017246   0.564109 -0.736322
30  60.254224  8.017246   0.564109 -0.736322
31  60.254224  8.017246   0.564109 -0.736322
52  60.254224  8.017246   0.564109 -0.736322
53  60.254224  8.017246   0.564109 -0.736322



/Users/alan/q-trade/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/alan/q-trade/alanq/optimization/factor_optimizer.py:580: RuntimeWarning: invalid value encountered in scalar divide
  scaled_value = (value - data_min) / (data_max - data_min)


In [2]:
best_config

{'buy_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20, 'skip_days': 10}]",
 'sell_factors': "[{'class': <class 'alanq.factors.timing.atr_stop_sell_factor.AtrStopSellFactor'>, 'atr_n': 1.0}]",
 '總得分': np.float64(60.2542238354455)}

# 多股投資組合最佳擇時(Timing)因子選擇
* 一般先用擇股(Selection)模組選出股票，再進行擇時

In [ ]:

import setup_path  # 設定 Python 路徑以引用 alanq 模組
from alanq.optimization import ParameterSpace, MultiStockFactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor
from alanq.data import StockDataManager

# 1. 準備多股票資料
data_manager = StockDataManager(["AAPL", "TSLA", "MSFT"], start_date="2020-01-01")
stock_data = {symbol: data_manager.get_kl_pd(symbol) for symbol in ["AAPL", "TSLA", "MSFT"]}

# 2. 定義參數空間
param_space = ParameterSpace()

# 添加買入因子
param_space.add_buy_factor(
    BreakoutBuyFactor,
    {'xd': [10, 60], 'skip_days': [5]}
)

# 添加賣出因子
param_space.add_sell_factor(
    BreakdownSellFactor,
    {'xd': [5]}
)
param_space.add_sell_factor(
    AtrStopSellFactor,
    {'atr_n': [1.0]}
)

custom_weights = {
        '策略_總報酬率': 1,     
        '策略_Sharpe': 1,       
        '策略_最大回撤': 3,            
        # 其他指標權重為 0（不考慮）
    }

# 3. 執行優化
optimizer = MultiStockFactorOptimizer(
        stock_data=stock_data,              # 必要：股票資料字典
        parameter_space=param_space,          # 必要：參數空間定義
        initial_capital=1_000_000,           # 可選：初始資金（預設 1,000,000）
        slippage_factors=None,   # 可選：滑價因子列表（預設 None）
        position_manager=None,    # 可選：倉位管理器（預設 None）
        show_progress=True,                  # 可選：顯示進度條（預設 True）
        n_jobs=-1,                          # 可選：並行進程數，-1=所有核心（預設 1）
        metric_weights=custom_weights,       # 可選：指標權重字典（預設 None=等權重）
        enable_full_rate_factor=False
    )

best_config, results_df = optimizer.optimize()

# 4. 查看結果
print(f"最佳配置: {best_config}")
print(f"前 10 名: {optimizer.get_top_n(10)}")

正在使用 yfinance 下載 3 檔股票資料...
------------------------------
已成功下載 3 檔股票資料
  - AAPL: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - MSFT: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
------------------------------
總共需要測試 6 種組合...
股票數量: 3
股票代號: ['AAPL', 'TSLA', 'MSFT']
使用 8 個進程進行並行處理...


收集指標數據: 100%|██████████| 6/6 [00:03<00:00,  1.74it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.6000
最佳配置: {'buy_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 60, 'skip_days': 5}]", 'sell_factors': "[{'class': <class 'alanq.factors.timing.atr_stop_sell_factor.AtrStopSellFactor'>, 'atr_n': 1.0}]", '總得分': np.float64(80.0)}
前 10 名:          總得分                                               買入因子  \
1  80.000000  [{'class': <class 'alanq.factors.timing.breako...   
0  73.422178  [{'class': <class 'alanq.factors.timing.breako...   
2  46.039605  [{'class': <class 'alanq.factors.timing.breako...   
5  46.039605  [{'class': <class 'alanq.factors.timing.breako...   
3   6.906796  [{'class': <class 'alanq.factors.timing.breako...   
4   6.906796  [{'class': <class 'alanq.factors.timing.breako...   

                                                賣出因子  策略_總報酬率_得分  策略_年化報酬率_得分  \
1  [{'class': <class 'alanq.factors.timing.atr_st...   20.000000         


/Users/alan/q-trade/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/alan/q-trade/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4674: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5,


In [30]:
results_df

,總得分,買入因子,賣出因子,策略_總報酬率_得分,策略_年化報酬率_得分,策略_Sharpe_得分,策略_最大回撤_得分,策略_年化波動率_得分,勝率_得分,盈虧比_得分,...,profit_loss_ratio,avg_profit,avg_loss,net_profit,max_single_profit,max_single_loss,avg_holding_days,avg_return,max_consecutive_wins,max_consecutive_losses
0,80.000000,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.atr_st...,20.000000,0,0.000000,60.000000,0,0,0,...,inf,993586.548430,0.000000,2.980760e+06,1.522097e+06,205200.464342,2005.333333,3.412305,3,0
1,73.422178,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.atr_st...,18.776278,0,0.553413,54.092487,0,0,0,...,inf,947944.806007,0.000000,2.843834e+06,1.215084e+06,632572.821169,2134.000000,4.747772,3,0
2,46.039605,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,15.837030,0,20.000000,10.202575,0,0,0,...,2.812989,69888.471784,24844.912486,2.514955e+06,4.972703e+05,-111513.481814,11.245000,0.019695,4,10
3,46.039605,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,15.837030,0,20.000000,10.202575,0,0,0,...,2.812989,69888.471784,24844.912486,2.514955e+06,4.972703e+05,-111513.481814,11.245000,0.019695,4,10
4,6.906796,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,0.000000,0,6.906796,0.000000,0,0,0,...,3.566753,67593.793214,18951.074465,7.429114e+05,3.645316e+05,-69292.605542,12.790698,0.029901,3,6
5,6.906796,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,0.000000,0,6.906796,0.000000,0,0,0,...,3.566753,67593.793214,18951.074465,7.429114e+05,3.645316e+05,-69292.605542,12.790698,0.029901,3,6


# MultiStockFactorOptimizer 的績效指標修改
* 將原先 custom_weights 的 dict[str, float] 結構轉換為績效指標結構：
```
{
    "class": BaseBasicMetric 或 BaseDetailMetric 的子類,
    "weight": float
}
```

In [ ]:
import setup_path  # 設定 Python 路徑以引用 alanq 模組
from alanq.performance import (
    TotalReturnMetric,
    SharpeMetric,
    MaxDrawdownMetric,
    WinRateMetric,
    ProfitLossRatioMetric,
    NetProfitMetric
)

# =========================================================
# 更新範例 ：多股票優化（使用指標類別權重）
# =========================================================
def example_multi_stock_optimization():
    """多股票優化，使用指標類別權重"""
    
    # 1. 準備多股票資料
    data_manager = StockDataManager(["AAPL", "TSLA", "MSFT"], start_date="2020-01-01")
    stock_data = {
        symbol: data_manager.get_kl_pd(symbol) 
        for symbol in ["AAPL", "TSLA", "MSFT"]
    }
    
    # 2. 定義參數空間
    param_space = ParameterSpace()
    
    # 添加買入因子
    param_space.add_buy_factor(
        BreakoutBuyFactor,
        {'xd': [10, 60], 'skip_days': [5]}
    )

    # 添加賣出因子
    param_space.add_sell_factor(
        BreakdownSellFactor,
        {'xd': [5]}
    )
    param_space.add_sell_factor(
        AtrStopSellFactor,
        {'atr_n': [1.0]}
    )
    
    # 3. 使用指標類別定義權重
    metric_weights = [
        {"class": TotalReturnMetric, "weight": 1},
        {"class": SharpeMetric, "weight": 1},        
        {"class": MaxDrawdownMetric, "weight": 3},
        # {"class": WinRateMetric, "weight": 1},
        # {"class": ProfitLossRatioMetric, "weight": 1},  
        # {"class": NetProfitMetric, "weight": 1},        
    ]
    
    # 4. 執行優化
    optimizer = MultiStockFactorOptimizer(
        stock_data=stock_data,
        parameter_space=param_space,
        initial_capital=1_000_000,
        metric_weights=metric_weights,  # 傳入指標類別列表
        show_progress=True,
        n_jobs=-1,  # 使用所有 CPU 核心
        enable_full_rate_factor=False
    )
    
    best_config, results_df = optimizer.optimize()
    
    return best_config, results_df


# =========================================================
# 範例 5：查看指標類別的屬性
# =========================================================
def example_view_metric_attributes():
    """查看指標類別的屬性"""
    
    print("指標類別屬性範例：")
    print("=" * 60)
    
    # 創建指標實例以查看屬性
    metrics_to_check = [
        TotalReturnMetric(),
        SharpeMetric(),
        MaxDrawdownMetric(),
        WinRateMetric(),
    ]
    
    for metric in metrics_to_check:
        print(f"\n{metric.__class__.__name__}")
        print(f"  指標名稱 (metric_name): {metric.metric_name}")
        print(f"  該指標是否越大越好 (higher_is_better): {metric.higher_is_better}")
        print(f"  目標值 (target) -> 用於單股backtester，已棄用: {metric.target}")
        print(f"  描述 (description): {metric.description}")



# 查看指標類別屬性
# example_view_metric_attributes()
best_config_performence_opt, results_df_performence_opt = example_multi_stock_optimization()

正在使用 yfinance 下載 3 檔股票資料...
------------------------------
已成功下載 3 檔股票資料
  - AAPL: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - MSFT: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
------------------------------
總共需要測試 6 種組合...
股票數量: 3
股票代號: ['AAPL', 'TSLA', 'MSFT']
使用 8 個進程進行並行處理...


收集指標數據: 100%|██████████| 6/6 [00:04<00:00,  1.26it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.6000



/Users/alan/q-trade/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/alan/q-trade/venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4674: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5,


In [32]:
# TEST: 看看相同參數不同使用方法是否得到相同的最佳參數
# best_config：使用dict[str, float] 權重
# best_config_performence_opt：使用指標類別權重
# 結果應該相同
print(best_config_performence_opt['buy_factors'] == best_config['buy_factors'])
print(best_config_performence_opt['sell_factors'] == best_config['sell_factors'])


True
True
